In [ ]:
import numpy as np
import pandas as pd

In [ ]:
dataDf = pd.read_csv('data.csv')
x = dataDf['LATITUDE'].tolist()
y = dataDf['LONGITUDE'].tolist()
T = dataDf['ALTITUDE'].tolist()

%matplotlib notebook

# x = [i for i in range(1,51)]
# y = [i for i in range(51,1,-1)]
# # T = [(x[i]*x[i]) + (y[i]*0) for i in range(50)]
# T = [(x[i]*0.6389) + (y[i]*1.253) for i in range(50)]
# T_ = [(x[i]*0.8436) + (y[i]*1.0483) + 0.20469999958167995 for i in range(50)]

# print(T_)
# print(T)

# from mpl_toolkits import mplot3d
# import matplotlib.pyplot as plt

# # plotting 1000 random points from the dataset

# # fig = plt.figure()
# # ax = fig.add_subplot(111, projection='3d')
# # ax1 = fig.add_subplot(111, projection='3d')
# xdata = x
# ydata = y
# # zdata = T_
# # ax = ax.scatter(xdata, ydata, zdata)
# zdata = T
# # ax1 = ax.scatter(xdata, ydata, zdata)
# # sample = dataDf.sample(n = 1000)

# ax = plt.subplots(subplot_kw={'projection': '3d'})
# ax.plot(xdata, ydata, zdata)

In [ ]:
def normalize(x):
    
    mean = np.mean(x)
    denominator = max(x) - min(y)
    x = (x - mean) / denominator
    
    return x

In [ ]:
def generate_features(x, y, N, deg):
    
    # the feature matrix
    # for each pow of y (dy), get all pow of x (dx), such that dx + dy = deg
    
    # [[1, x1, x1^2, y1, x1y1, y1^2]
    #  [1, x2, x2^2, y2, x2y2, y2^2]
    #   .  .
    #   .  .
    #  [1, xN, xN^2, yN, xNyN, yN^2]]
    featureMatrix = []
    
    # number of features
    d = 0
    
    for n in range(N):
        row = []
        for i in range(deg + 1):
            for j in range(deg - i + 1):
                term = (x[n]**j) * (y[n]**i)
                row.append(term)
                if n == 0:
                    d += 1
        featureMatrix.append(row)
    
    # converting to a numpy array
    featureMatrix = np.array(featureMatrix)
    
    return featureMatrix, d

In [ ]:
def update_weights(W, features, error, eta):
    
    delta = features.dot(error)
    # print(features)
    # print(error)
    # print(delta)
    # print()
    W = W - eta*delta
    
    return W

In [ ]:
def show(W, error, d):
    
    for i in range(d):
        print(W[i])
    print('Error: ', error)
    print('\n')

In [ ]:
def generate_model(x, y, T, deg, maxIter, eta):
    
    import random
    from tqdm import tqdm_notebook
       
    N = len(x)
    featureMatrix, d = generate_features(x, y, N, deg)
    
    # initial weights vector
    # W = np.array([0 for i in range(d)])
    random.seed(12)
    W = np.array([random.random() for i in range(d)])
    
    currentError = 0.5 * (sum(T)**2)
    
    for i in tqdm_notebook(range(maxIter)):
        
        H = featureMatrix.dot(W)
        E = 0.5 * (H - T) * np.transpose(H - T)
        prevError = currentError
        currentError = sum(E)
        show(W, currentError, d)
        W = update_weights(W, np.transpose(featureMatrix), H - T, eta)
        
    return W

In [ ]:
generate_model(x, y, T, 1, 10000, 0.000000001)

# Unnormalized

## Max iterations = 10,000
## eta = 0.000000001

### Degree = 2

0.4744234599520871      1 <br>
0.6474382918705535      x <br>
0.0005804929584987999   x^2 <br>
0.14105769027351664     y <br>
-0.09403124677459616    xy <br>
0.3544625440157649      y^2 <br>

Error:  75436974.66365339

---------------------------------------------------

### Degree = 1

0.5185508389987425       1 <br>
0.15099146093306134      x <br>
1.3394518499352581       y <br>

Error:  75282294.06572284